In [1]:
# import library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from feature_engine.encoding import OneHotEncoder
import string


/opt/anaconda3/envs/machine_learnig_gomila/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


---
## Preprocesado
---

In [2]:
data = pd.read_csv('../Data/data_preproce.csv')

In [4]:
data.head()

,variation,verified_reviews,feedback
0,Charcoal Fabric,Love my Echo!,1
1,Charcoal Fabric,Loved it!,1
2,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,Charcoal Fabric,Music,1


In [66]:
one_enc = OneHotEncoder(variables=['variation'],drop_last=True) # k-1
data_enc = one_enc.fit_transform(data)

In [6]:
data_enc.head()

,verified_reviews,feedback,variation_Charcoal Fabric,variation_Walnut Finish,variation_Heather Gray Fabric,variation_Sandstone Fabric,variation_Oak Finish,variation_Black,variation_White,variation_Black Spot,variation_White Spot,variation_Black Show,variation_White Show,variation_Black Plus,variation_White Plus,variation_Configuration: Fire TV Stick,variation_Black Dot
0,Love my Echo!,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Loved it!,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Sometimes while playing a game, you can answer...",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,I have had a lot of fun with this thing. My 4 ...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Music,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
#eliminemos los signos de puntuación
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andres/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
len(stopwords.words('english'))

179

In [39]:
def clear_words(frase):
  review = re.sub('[^a-zA-Z]', ' ', frase)
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if  word not in set(all_stopwords)]
  review = ' '.join(review)
  return review

In [71]:
df_temp = data_enc['verified_reviews'].apply(clear_words)

In [44]:
# vectorizacion de palabras
from sklearn.feature_extraction.text import CountVectorizer

In [72]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(df_temp)

In [75]:
vectorizer.get_feature_names_out()[:10]

array(['abay', 'abc', 'abd', 'abil', 'abl', 'abod', 'absolut',
       'absolutli', 'ac', 'accent'], dtype=object)

In [74]:
x.toarray().shape

(3149, 2766)

In [76]:
data_enc.drop(columns='verified_reviews',inplace=True)
df_temp = pd.DataFrame(x.toarray())
data_enc = pd.concat([data_enc,df_temp],ignore_index=True,axis=1)
data_enc.head()

,0,1,2,3,4,5,6,7,8,9,...,2772,2773,2774,2775,2776,2777,2778,2779,2780,2781
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---
**Modelo Naive Bayes**

---